# Hybrid Recommendation System using SVD

This notebook implements a hybrid recommendation system using Singular Value Decomposition (SVD), which combines collaborative filtering and content-based approaches for the educational content recommendation system.

## Objectives
1. Prepare user-item interaction matrix
2. Extract item features for content-based filtering
3. Implement collaborative filtering using SVD
4. Combine with content-based approach
5. Evaluate and tune the hybrid model

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse.linalg import svds
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import pickle
import os
import time
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, mean_squared_error

# For reproducibility
np.random.seed(42)

# Configure plotting
sns.set_theme(style="whitegrid", palette="viridis")

## 1. Load and Prepare Data

In [3]:
# Load the datasets
lectures_data = pd.read_csv('../data/cleaned/cleaned_lectures.csv')
merged_data = pd.read_csv('../data/cleaned/merged_cleaned_data.csv')

# Display basic info about the datasets
print(f"Lectures dataset shape: {lectures_data.shape}")
print(f"Merged data shape: {merged_data.shape}")

Lectures dataset shape: (1021, 6)
Merged data shape: (117167, 14)


In [5]:
# Create subject categories based on ranges of tag values
# These mappings are based on TOEIC structure
def get_subject_category(tag):
    try:
        tag_val = float(tag)
        if 1 <= tag_val < 23:
            return "Listening Skills"
        elif 23 <= tag_val < 52:
            return "Reading Skills"
        elif 52 <= tag_val < 70:
            return "Speaking Skills"
        elif 70 <= tag_val < 150:
            return "Writing Skills"
        elif 150 <= tag_val < 200:
            return "Test Preparation"
        elif 200 <= tag_val < 300:
            return "Grammar & Vocabulary"
        else:
            return "General"
    except:
        return "General"

# Map part numbers to human-readable names
part_names = {
    0: "Introduction",
    1: "Listening Comprehension",
    2: "Reading Comprehension",
    3: "Grammar & Vocabulary",
    4: "Speaking Assessment",
    5: "Writing Exercises",
    6: "Practice Tests",
    7: "Additional Resources"
}

# Apply mappings
lectures_data['subject_category'] = lectures_data['tags'].apply(get_subject_category)
lectures_data['part_name'] = lectures_data['part'].map(part_names)

## 2. Create Bundle Features

Let's extract bundle information for content-based features in our hybrid model.

In [6]:
# Extract unique bundle information
bundle_info = merged_data.groupby('bundle_id').agg({
    'part': 'first',
    'tags': lambda x: ';'.join(set(str(i) for i in x if pd.notna(i))),
    'question_id': lambda x: len(set(x))  # Number of questions in bundle
}).reset_index()

# Rename columns for clarity
bundle_info.columns = ['bundle_id', 'part', 'tags', 'question_count']

# Map part to human-readable names
bundle_info['part_name'] = bundle_info['part'].map(part_names)

# Create subject category based on tags
bundle_info['subject_category'] = bundle_info['tags'].apply(get_subject_category)

# Calculate bundle popularity from interaction data
bundle_popularity = merged_data['bundle_id'].value_counts().reset_index()
bundle_popularity.columns = ['bundle_id', 'interaction_count']

# Calculate bundle difficulty from correct answer rates
bundle_difficulty = merged_data.groupby('bundle_id').apply(
    lambda x: (x['user_answer'] == x['correct_answer']).mean()
).reset_index()
bundle_difficulty.columns = ['bundle_id', 'success_rate']

# Merge all features
bundle_features = bundle_info.merge(bundle_popularity, on='bundle_id', how='left')
bundle_features = bundle_features.merge(bundle_difficulty, on='bundle_id', how='left')

# Fill missing values
bundle_features['interaction_count'] = bundle_features['interaction_count'].fillna(0)
bundle_features['success_rate'] = bundle_features['success_rate'].fillna(0.5)

print(f"Total unique bundles: {len(bundle_features)}")
bundle_features.head()

Total unique bundles: 8260


C:\Users\karat\AppData\Local\Temp\ipykernel_17696\3732587692.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bundle_difficulty = merged_data.groupby('bundle_id').apply(


,bundle_id,part,tags,question_count,part_name,subject_category,interaction_count,success_rate
0,b1,1,1;2;179;181,1,Listening Comprehension,General,6,0.833333
1,b10,1,17;7;182,1,Listening Comprehension,General,47,0.319149
2,b100,1,22;2;181,1,Listening Comprehension,General,7,1.000000
3,b1000,2,24;33;182;183,1,Reading Comprehension,General,46,0.760870
4,b1001,2,34;35;182;183,1,Reading Comprehension,General,12,0.583333


## 3. Create User-Item Interaction Matrix

In [7]:
# Filter users with minimum interactions for better model performance
min_interactions = 5
user_counts = merged_data['user_id'].value_counts()
valid_users = user_counts[user_counts >= min_interactions].index
filtered_data = merged_data[merged_data['user_id'].isin(valid_users)]

print(f"Original users: {merged_data['user_id'].nunique()}")
print(f"Filtered users with {min_interactions}+ interactions: {len(valid_users)}")
print(f"Retained {len(filtered_data) / len(merged_data):.2%} of interactions")

Original users: 999
Filtered users with 5+ interactions: 881
Retained 99.74% of interactions


In [8]:
# Create correctness as a measure of interaction quality
filtered_data = filtered_data.copy()  # Ensures modifications don't affect original data
filtered_data.loc[:, 'correct'] = (filtered_data['user_answer'] == filtered_data['correct_answer']).astype(int)

# Aggregate user-item interactions
user_item_data = filtered_data.groupby(['user_id', 'bundle_id']).agg(
    correctness_rate=('correct', 'mean'),
    interaction_count=('correct', 'count'),
    avg_time=('elapsed_time', 'mean'),
    part=('part', 'first'),
    tags=('tags', lambda x: ';'.join(set(str(i) for i in x)))
).reset_index()

# Create pivot table with interaction count as values
user_item_matrix = user_item_data.pivot_table(
    index='user_id', 
    columns='bundle_id', 
    values='interaction_count',
    fill_value=0
)

# Display user-item matrix characteristics
print(f"User-item matrix shape: {user_item_matrix.shape}")
print(f"Sparsity: {1 - (user_item_matrix > 0).sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]):.4f}")

User-item matrix shape: (881, 8256)
Sparsity: 0.9892


## 4. Create ID Mappings

In [9]:
# Create user and bundle ID mappings
user_ids = filtered_data['user_id'].unique()
bundle_ids = filtered_data['bundle_id'].unique()

# Create user mapping
user_map = {user_id: idx for idx, user_id in enumerate(np.unique(user_ids))}
reverse_user_map = {idx: user_id for user_id, idx in user_map.items()}

# Create item mapping
item_map = {bundle_id: idx for idx, bundle_id in enumerate(np.unique(bundle_ids))}
reverse_item_map = {idx: bundle_id for bundle_id, idx in item_map.items()}

print(f"Number of users: {len(user_map)}")
print(f"Number of items: {len(item_map)}")

Number of users: 881
Number of items: 8256


## 5. Prepare Content Features Matrix

In [12]:
# Create feature vectors for content-based part
n_items = len(item_map)

# Add difficulty category
bundle_features['difficulty_level'] = pd.cut(
    bundle_features['success_rate'], 
    bins=[0, 0.3, 0.7, 1.0], 
    labels=['Hard', 'Medium', 'Easy']
)

# One-hot encode features
part_dummies = pd.get_dummies(bundle_features['part'], prefix='part')
subject_dummies = pd.get_dummies(bundle_features['subject_category'], prefix='subject')
difficulty_dummies = pd.get_dummies(bundle_features['difficulty_level'], prefix='difficulty')

# Combine feature matrices
content_features = pd.concat([part_dummies, subject_dummies, difficulty_dummies], axis=1)

# Add bundle_id for mapping
content_features = pd.concat([bundle_features[['bundle_id']], content_features], axis=1)

# Create a matrix with internal indices
# feature_matrix = np.zeros((n_items, content_features.shape[1] - 1))
feature_matrix = np.zeros((n_items, content_features.drop(columns=['bundle_id']).shape[1]))

# Fill the feature matrix with values
for _, row in content_features.iterrows():
    bundle_id = row['bundle_id']
    if bundle_id in item_map:
        idx = item_map[bundle_id]
        feature_matrix[idx] = row.values[1:]

# Normalize features
feature_matrix = normalize(feature_matrix, axis=1, norm='l2')

# Compute content similarity
content_similarity = cosine_similarity(feature_matrix)

print(f"Content features matrix shape: {feature_matrix.shape}")

Content features matrix shape: (8256, 12)


## 6. Create Ratings Matrix for SVD

In [13]:
# Create ratings matrix for SVD
n_users = len(user_map)
n_items = len(item_map)
ratings_matrix = np.zeros((n_users, n_items))

# Fill with interaction values
for _, row in user_item_data.iterrows():
    user_id = row['user_id']
    bundle_id = row['bundle_id']
    
    if user_id in user_map and bundle_id in item_map:
        user_idx = user_map[user_id]
        item_idx = item_map[bundle_id]
        
        # Use interaction_count as the rating
        rating = row['interaction_count']
        
        # Additionally, boost rating if correctness rate is high
        if not pd.isna(row['correctness_rate']):
            # Scale correctness rate to a small boost (0.1-0.5)
            boost = 0.1 + (row['correctness_rate'] * 0.4)
            rating = rating * (1 + boost)
        
        ratings_matrix[user_idx, item_idx] = rating

# Calculate mean rating of non-zero entries
mean_rating = np.mean(ratings_matrix[ratings_matrix > 0])
print(f"Ratings matrix shape: {ratings_matrix.shape}")
print(f"Mean rating: {mean_rating:.4f}")

Ratings matrix shape: (881, 8256)
Mean rating: 2.0131


## 7. Train-Test Split for Evaluation

In [14]:
# Create a list of all non-zero entries in the ratings matrix
interactions = []
for u in range(n_users):
    for i in range(n_items):
        if ratings_matrix[u, i] > 0:
            interactions.append((u, i, ratings_matrix[u, i]))

# Split interactions into train and test sets
train_interactions, test_interactions = train_test_split(
    interactions, test_size=0.2, random_state=42
)

# Create train and test matrices
train_matrix = np.zeros_like(ratings_matrix)
test_matrix = np.zeros_like(ratings_matrix)

# Fill train matrix
for u, i, r in train_interactions:
    train_matrix[u, i] = r

# Fill test matrix
for u, i, r in test_interactions:
    test_matrix[u, i] = r

print(f"Total interactions: {len(interactions):,}")
print(f"Training set: {len(train_interactions):,} interactions")
print(f"Test set: {len(test_interactions):,} interactions")

Total interactions: 78,656
Training set: 62,924 interactions
Test set: 15,732 interactions


## 8. SVD Model Implementation

In [ ]:
class SVDHybridRecommender:
    """SVD-based hybrid recommender system"""
    
    def __init__(self, n_factors=20, combine_weight=0.7):
        """
        Initialize the recommender.
        
        Args:
            n_factors: Number of latent factors for SVD
            combine_weight: Weight for combining CF and CB scores (0-1),
                           higher means more weight to CF
        """
        self.n_factors = n_factors
        self.combine_weight = combine_weight
        self.user_factors = None
        self.item_factors = None
        self.content_similarity = None
        self.mean_rating = None
    
    def fit(self, ratings_matrix, content_similarity=None):
        """
        Train the recommender.
        
        Args:
            ratings_matrix: User-item ratings matrix
            content_similarity: Item-item content similarity matrix
        """
        # Store content similarity if provided
        self.content_similarity = content_similarity
        
        # Calculate mean rating
        self.mean_rating = np.mean(ratings_matrix[ratings_matrix > 0])
        
        # Fill missing values with mean
        ratings_filled = ratings_matrix.copy()
        ratings_filled[ratings_filled == 0] = self.mean_rating
        
        # Apply SVD
        start_time = time.time()
        u, sigma, vt = svds(ratings_filled, k=min(self.n_factors, min(ratings_filled.shape) - 1))
        end_time = time.time()
        
        # Convert to latent factor matrices
        sigma_diag = np.diag(sigma)
        self.user_factors = u
        self.item_factors = vt.T
        
        print(f"SVD model trained in {end_time - start_time:.2f} seconds")
        print(f"User factors shape: {self.user_factors.shape}")
        print(f"Item factors shape: {self.item_factors.shape}")
    
    def predict_rating(self, user_idx, item_idx):
        """
        Predict rating for a user-item pair.
        
        Args:
            user_idx: User index
            item_idx: Item index
            
        Returns:
            Predicted rating
        """
        if self.user_factors is None or self.item_factors is None:
            return self.mean_rating
        
        return np.dot(self.user_factors[user_idx], self.item_factors[item_idx])
    
    def _get_content_score(self, known_items, item_idx):
        """
        Get content-based score for an item based on items the user has interacted with.
        
        Args:
            known_items: List of item indices the user has interacted with
            item_idx: Target item index
            
        Returns:
            Content-based similarity score
        """
        if self.content_similarity is None or len(known_items) == 0:
            return 0.0
        
        # Calculate average similarity to known items
        return np.mean([self.content_similarity[item_idx, idx] for idx in known_items])
    
   # ...existing code...
    def get_recommendations(self, user_idx, n=10, exclude_seen=True, known_items=None):
        """
        Generate recommendations for a user.
        ...
        """
        if self.user_factors is None or self.item_factors is None:
            return []
        
        # Get collaborative filtering scores
        cf_scores = np.dot(self.user_factors[user_idx], self.item_factors.T)
        
        # Initialize final scores
        final_scores = cf_scores.copy()
        
        # Combine with content-based scores if available
        if self.content_similarity is not None and known_items is not None and len(known_items) > 0:
            for i in range(len(final_scores)):
                cb_score = self._get_content_score(known_items, i)
                final_scores[i] = (self.combine_weight * cf_scores[i]) + ((1 - self.combine_weight) * cb_score)
        
        # Get indices of all items
        all_items = np.arange(len(final_scores))
        
        # Exclude seen items if requested
        if exclude_seen and known_items is not None and len(known_items) > 0:
            mask = np.ones_like(final_scores, dtype=bool)
            mask[known_items] = False
            all_items = all_items[mask]
            final_scores = final_scores[mask]
        
        # Sort by score and get top N
        top_indices = np.argsort(-final_scores)[:n]
        
        # Get actual item indices
        top_items = all_items[top_indices]
        top_scores = final_scores[top_indices]
        
        return list(zip(top_items, top_scores))

# Initialize and train the SVD hybrid recommender
svd_recommender = SVDHybridRecommender(n_factors=20, combine_weight=0.7)
svd_recommender.fit(train_matrix, content_similarity)

SVD model trained in 0.17 seconds
User factors shape: (881, 20)
Item factors shape: (8256, 20)


## 9. Evaluate the Model

In [16]:
# Calculate RMSE on test data
def calculate_rmse(model, test_matrix):
    """
    Calculate Root Mean Squared Error for a model on the given test data.
    
    Parameters:
    model : Trained recommendation model with a predict_rating(user, item) method
    test_matrix : 2D NumPy array or sparse matrix with test ratings

    Returns:
    float : RMSE value
    """
    non_zero_indices = np.where(test_matrix > 0)
    y_true = []
    y_pred = []

    for u, i in zip(non_zero_indices[0], non_zero_indices[1]):
        y_true.append(test_matrix[u, i])
        y_pred.append(model.predict_rating(u, i))

    return np.sqrt(mean_squared_error(y_true, y_pred))

# Calculate precision and recall at k
def calculate_precision_recall_at_k(model, test_matrix, k=10):
    """
    Compute average precision@k and recall@k for all users in test set.

    Parameters:
    model : Trained recommendation model with get_recommendations(user, n, exclude_seen, known_items)
    test_matrix : 2D NumPy array with ground truth test interactions
    k : int : Number of top recommendations

    Returns:
    Tuple[float, float] : (Average precision@k, Average recall@k)
    """
    precision_sum = 0.0
    recall_sum = 0.0
    user_count = 0

    for u in range(test_matrix.shape[0]):
        test_items = np.where(test_matrix[u] > 0)[0]
        if len(test_items) == 0:
            continue

        known_items = np.where(train_matrix[u] > 0)[0].tolist()
        recs = model.get_recommendations(u, n=k, exclude_seen=True, known_items=known_items)
        if not recs:
            continue

        rec_items = [item for item, _ in recs]
        hits = len(set(rec_items) & set(test_items))

        precision_sum += hits / min(k, len(rec_items))
        recall_sum += hits / len(test_items)
        user_count += 1

    avg_precision = precision_sum / user_count if user_count > 0 else 0.0
    avg_recall = recall_sum / user_count if user_count > 0 else 0.0
    return avg_precision, avg_recall

# Evaluate the model
rmse = calculate_rmse(svd_recommender, test_matrix)
print(f"RMSE: {rmse:.4f}")

print("Calculating precision@10 and recall@10...")
precision, recall = calculate_precision_recall_at_k(svd_recommender, test_matrix, k=10)
print(f"Precision@10: {precision:.4f}")
print(f"Recall@10: {recall:.4f}")

RMSE: 2.5306
Calculating precision@10 and recall@10...
Precision@10: 0.0194
Recall@10: 0.0140


## 10. Generate Sample Recommendations

In [17]:
print(list(user_map.keys())[:5])  # See a few valid user IDs

['u100200', 'u100767', 'u101324', 'u101372', 'u102298']


In [18]:
# Generate recommendations for a user
def get_recommendations_for_user(user_id, model, n=10):
    """Generate recommendations for a user."""
    # Get user index
    if user_id not in user_map:
        return []
    
    user_idx = user_map[user_id]
    
    # Get known items
    known_items = np.where(train_matrix[user_idx] > 0)[0]
    
    # Generate recommendations
    recs = model.get_recommendations(user_idx, n=n, exclude_seen=True, known_items=known_items)
    
    # Convert to original IDs
    recommendations = [(reverse_item_map[item], score) for item, score in recs]
    
    return recommendations

# Get recommendations for a sample user
sample_user_id = "u100200"
recommendations = get_recommendations_for_user(sample_user_id, svd_recommender, n=10)

print(f"Recommendations for user {sample_user_id}:")
for i, (bundle_id, score) in enumerate(recommendations, 1):
    print(f"{i}. {bundle_id} (Score: {score:.2f})")

Recommendations for user u100200:
1. b5 (Score: 0.24)
2. b5567 (Score: 0.24)
3. b5579 (Score: 0.24)
4. b160 (Score: 0.24)
5. b192 (Score: 0.24)
6. b5584 (Score: 0.24)
7. b89 (Score: 0.24)
8. b190 (Score: 0.24)
9. b173 (Score: 0.24)
10. b5577 (Score: 0.24)


## 11. Compare with Pure SVD (No Content Features)

In [19]:
# Train a pure SVD model (without content features)
pure_svd = SVDHybridRecommender(n_factors=20, combine_weight=1.0)  # Weight of 1.0 means 100% CF
pure_svd.fit(train_matrix)

# Evaluate pure SVD
pure_rmse = calculate_rmse(pure_svd, test_matrix)
print(f"Pure SVD RMSE: {pure_rmse:.4f}")

print("Calculating precision@10 and recall@10...")
pure_precision, pure_recall = calculate_precision_recall_at_k(pure_svd, test_matrix, k=10)
print(f"Pure SVD Precision@10: {pure_precision:.4f}")
print(f"Pure SVD Recall@10: {pure_recall:.4f}")

# Print comparison
print("\nComparison:")
print("Method      | RMSE    | Precision@10 | Recall@10")
print("------------|---------|--------------|----------")
print(f"Hybrid SVD  | {rmse:.4f}  | {precision:.4f}       | {recall:.4f}")
print(f"Pure SVD    | {pure_rmse:.4f}  | {pure_precision:.4f}       | {pure_recall:.4f}")
print(f"Improvement | {pure_rmse-rmse:.4f}  | {precision-pure_precision:.4f}       | {recall-pure_recall:.4f}")

SVD model trained in 0.16 seconds
User factors shape: (881, 20)
Item factors shape: (8256, 20)
Pure SVD RMSE: 2.5306
Calculating precision@10 and recall@10...
Pure SVD Precision@10: 0.0439
Pure SVD Recall@10: 0.0735

Comparison:
Method      | RMSE    | Precision@10 | Recall@10
------------|---------|--------------|----------
Hybrid SVD  | 2.5306  | 0.0194       | 0.0140
Pure SVD    | 2.5306  | 0.0439       | 0.0735
Improvement | 0.0000  | -0.0246       | -0.0594


## 12. Save the Model

In [20]:
# Create directory if it doesn't exist
os.makedirs("../models", exist_ok=True)

# Create a model package with all necessary components
model_package = {
    'recommender': svd_recommender,
    'user_map': user_map,
    'item_map': item_map,
    'reverse_user_map': reverse_user_map,
    'reverse_item_map': reverse_item_map,
    'metrics': {
        'rmse': rmse,
        'precision@10': precision,
        'recall@10': recall
    }
}

# Save the model
with open("../models/svd_hybrid_model.pkl", "wb") as f:
    pickle.dump(model_package, f)

print("Model saved to ../models/svd_hybrid_model.pkl")

Model saved to ../models/svd_hybrid_model.pkl
